<a href="https://colab.research.google.com/github/verma-saloni/Thesis-Work/blob/main/Clean_Fasttext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install unidecode
!pip install word2number
!pip install contractions

In [ ]:
import pandas as pd
import numpy as np
import nltk
import gensim
from gensim.models import Word2Vec
import string
string.punctuation
nltk.download('stopwords')
import re
from bs4 import BeautifulSoup
import spacy
import unidecode
from word2number import w2n
import contractions
from nltk.tokenize.treebank import TreebankWordDetokenizer
# load spacy model, can be "en_core_web_sm" as well
from sklearn.model_selection import train_test_split
import xgboost as xgb



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from google.colab import drive
drive._mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_fake = pd.read_csv('/content/drive/MyDrive/Thesis/gossipcop_fake.csv')
df_real = pd.read_csv('/content/drive/MyDrive/Thesis/gossipcop_real.csv')
df_fake['labelML']=0
df_real['labelML']=1

df_fake.count()


id           5323
news_url     5067
title        5323
tweet_ids    5135
labelML      5323
dtype: int64

In [ ]:
dforiginal = df_fake.append(df_real)

In [ ]:
df= df_fake.append(df_real)
df.reset_index(drop=True, inplace=True)

In [ ]:
print(df.columns)
df.count()

Index(['id', 'news_url', 'title', 'tweet_ids', 'labelML'], dtype='object')


id           22140
news_url     21871
title        22140
tweet_ids    20894
labelML      22140
dtype: int64

In [ ]:
print(df)

                         id  ... labelML
0      gossipcop-2493749932  ...       0
1      gossipcop-4580247171  ...       0
2       gossipcop-941805037  ...       0
3      gossipcop-2547891536  ...       0
4      gossipcop-5476631226  ...       0
...                     ...  ...     ...
22135      gossipcop-891749  ...       1
22136      gossipcop-941486  ...       1
22137      gossipcop-953143  ...       1
22138      gossipcop-954366  ...       1
22139      gossipcop-877062  ...       1

[22140 rows x 5 columns]


In [ ]:
df1= df.drop(['tweet_ids', 'news_url','id'], axis=1)
df1.count()

title      22140
labelML    22140
dtype: int64

In [ ]:
# NLP Preprocessing
from gensim.utils import simple_preprocess

# NLP Preprocess - gensim.utils.simple_preprocess(doc, deacc=False, min_len=2, max_len=15)[source]
# Convert a document into a list of tokens.
# This lowercases, tokenizes, de-accents (optional). – the output are final tokens = unicode strings, that won’t be processed any further.

df1.iloc[:, 0] = df1.iloc[:, 0].apply(lambda x: ' '.join(simple_preprocess(x)))

In [ ]:
#print(df1)
df1.count()

title      22140
labelML    22140
dtype: int64

In [ ]:
# Prefixing each row of the category column with '__label__' 0;10 all rows. same for column. 
df1.iloc[:5322, 0] = df1.iloc[:, 0].apply(lambda x: '__label__0 ' + x)
df1.iloc[5323:, 0] = df1.iloc[1:, 0].apply(lambda x: '__label__1 ' + x)

# NOTE BIG NOTE In general it is important to properly preprocess your data. In particular our example scripts in the root folder do this. i can see why the last cell is wierd, fix in post.


In [ ]:
print(df1)

                                                   title  labelML
0      __label__0 did miley cyrus and liam hemsworth ...        0
1      __label__0 paris jackson cara delevingne enjoy...        0
2      __label__0 celebrities join tax march in prote...        0
3      __label__0 cindy crawford daughter kaia gerber...        0
4      __label__0 full list of oscar nominations variety        0
...                                                  ...      ...
22135  __label__1 hollywood film awards the complete ...        1
22136  __label__1 jada pinkett smith explains why son...        1
22137  __label__1 tinsley mortimer reacts to luann de...        1
22138  __label__1 prince harry carries on princess di...        1
22139  __label__1 kylie jenner is actually terrified ...        1

[22140 rows x 2 columns]


In [ ]:
df1.count()

title      22140
labelML    22140
dtype: int64

In [ ]:
df2=df1
# ------------- df2 = df2.reindex(columns=['labelML', 'title'])
#print(df1)

In [ ]:
traindf, testdf = train_test_split(df2, test_size=0.2)
print(traindf)
print(testdf)  

traindf.count()

                                                   title  labelML
14640  __label__1 the last ship takes production hiat...        1
21777  __label__1 sofia richie wears sky high balenci...        1
15456  __label__1 kim kardashian announces all dash s...        1
8859   __label__1 foo fighters james corden cover nev...        1
21236  __label__1 tom holland spills how he got in sh...        1
...                                                  ...      ...
2371   __label__0 fergie cries while talking about jo...        0
8649   __label__1 broke girls canceled at cbs after s...        1
16827  __label__1 saturday savings bella hadid ankle ...        1
11977  __label__1 shaun threatens to back out of help...        1
4892   __label__0 reports suggest miley cyrus and lia...        0

[17712 rows x 2 columns]
                                                   title  labelML
3377   __label__0 meghan and harry set to travel to c...        0
7690   __label__1 did richard pryor and marlon bra

title      17712
labelML    17712
dtype: int64

In [ ]:
# Saving the CSV file as a text file to train/test the classifier
import csv #needed
traindf[['labelML', 'title']].to_csv('traindf.txt', 
                                          index = False, 
                                          sep = ' ',
                                          header = None, 
                                          quoting = csv.QUOTE_NONE, 
                                          quotechar = "", 
                                          escapechar = " ")

testdf[['labelML', 'title']].to_csv('testdf.txt', 
                                     index = False, 
                                     sep = ' ',
                                     header = None, 
                                     quoting = csv.QUOTE_NONE, 
                                     quotechar = "", 
                                     escapechar = " ")

In [ ]:
!pip install fasttext
import fasttext
# Training the fastText classifier
model = fasttext.train_supervised('traindf.txt', wordNgrams = 2, dim=300)

# Evaluating performance on the entire test file
model.test('testdf.txt')                      

# Predicting on a single input
testdf['labelML'] = testdf['labelML'].astype(str)
model.predict(testdf.iloc[56, 0])
#model.test(testdf.txt)

# Save the trained model
#model.save_model('modelFTme.bin')

(('__label__1',), array([0.77910531]))

In [ ]:
testdf.head(10)
#print(testdf.columns)

,title,labelML
3377,__label__0 meghan and harry set to travel to c...,0
7690,__label__1 did richard pryor and marlon brando...,1
7685,__label__1 kate middleton takes more serious r...,1
3244,__label__0 liam payne begging cheryl to move l...,0
17445,__label__1 kylie jenner talks post baby body i...,1
4874,__label__0 are rihanna bf hassan jameel talkin...,0
18111,__label__1 mischa barton opens up to amanda kn...,1
16528,__label__1 jonathan rhys meyers reunites with ...,1
9781,__label__1 jamie oliver wants to end his long ...,1
15992,__label__1 watch blue ivy carter casually drop...,1


This model is running, so trying to change with vectors, or textual input for new vectors. Accuracy is only 50 percent coz I've given it such a balanced dataset.

In [ ]:
# Predicting on a single input
#model.predict(test.iloc['Miley cyrus is new'])
#model.test(testdf.txt)
#use this later when we have the representations I guess. 


In [ ]:
# Once the model is trained, we can retrieve the list of words and labels

# model.words 
model.labels
#It returns all words in the vocabulary, sorted by decreasing frequency. 

['__label__1', '__label__0']

In [ ]:
# To evaluate our model by computing the precision at 1 (P@1) and the recall on a test set, we use the test function:

def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

print_results(*model.test('testdf.txt'))

N	4428
P@1	1.000
R@1	1.000


In [ ]:
#We can also predict labels for a specific text. By default, predict returns only one label :
# the one with the highest probability. You can also predict more than one label by specifying the parameter k:
model.predict("katy perry and orlando bloom cozy up", k=2)

(('__label__1', '__label__0'), array([0.69645387, 0.30356613]))

In [ ]:
#traindf['title'] = 
model.get_word_vector("the")

array([ 2.09438428e-03,  5.61120585e-02, -3.51719931e-02, -3.89494188e-02,
        2.44774204e-02,  4.08992358e-02, -2.04787608e-02, -1.04940534e-02,
        4.04046103e-02,  6.25133738e-02, -1.01860054e-02,  5.73546849e-02,
       -2.32897680e-02,  3.78556065e-02,  1.56504624e-02,  1.95619348e-03,
       -2.44491063e-02, -1.10713998e-02,  2.32058261e-02, -7.49038765e-03,
        4.56989072e-02,  5.62664084e-02,  1.74952764e-02, -1.60713196e-02,
       -1.97066902e-03,  7.74711976e-03,  2.67540943e-02, -5.14450064e-03,
       -5.30177876e-02,  2.88933124e-02,  3.51334456e-03,  2.18358394e-02,
       -3.61335501e-02,  2.64903512e-02, -2.42466889e-02,  3.13576236e-02,
        2.71694530e-02, -4.76960912e-02, -2.01985184e-02, -5.37317665e-03,
        2.63489764e-02,  3.15811373e-02,  2.87264516e-03, -2.72718370e-02,
        8.17730557e-04, -2.75171623e-02,  1.85051411e-02, -1.21043362e-02,
       -6.12276315e-04,  2.43964754e-02, -3.49434689e-02,  1.75606851e-02,
       -2.35582441e-02, -

In [ ]:
model.get_sentence_vector("simon cowell rushed to hospital")

array([ 3.5500148e-04,  6.4496002e-03, -5.0753667e-03, -3.7310808e-03,
        3.1935279e-03,  4.4048806e-03, -2.4639056e-03, -1.5988834e-03,
        4.4415891e-03,  6.7489841e-03, -3.8804268e-04,  7.1050995e-03,
       -2.1653159e-03,  5.3569288e-03,  2.2348780e-03, -6.3896627e-04,
       -2.5260511e-03, -5.4242584e-04,  1.5904907e-03, -1.7656873e-03,
        5.1544593e-03,  5.9493668e-03,  1.2310349e-03, -1.3424666e-03,
       -5.6071905e-04,  4.6616903e-04,  4.0101106e-03,  6.1338324e-05,
       -7.3400019e-03,  3.1253023e-03,  6.2540686e-04,  3.2736536e-03,
       -3.7403058e-03,  2.7353130e-03, -3.3486474e-03,  3.5302369e-03,
        2.8328935e-03, -6.4758426e-03, -1.6563958e-03,  4.5332516e-05,
        3.1899733e-03,  3.9490405e-03,  5.5276277e-04, -3.1335070e-03,
       -5.0412101e-04, -2.8030700e-03,  1.4306280e-03, -1.4156816e-03,
       -4.2882617e-04,  3.4061936e-03, -4.1652056e-03,  2.1724566e-03,
       -2.8463621e-03, -1.5721882e-03, -2.2739451e-04, -5.6600370e-03,
      

In [ ]:
model.get_nearest_neighbors('green') #https://fasttext.cc/docs/en/unsupervised-tutorial.html

[(0.5160191655158997, 'so'),
 (0.5104241967201233, 'the'),
 (0.510277509689331, 'awards'),
 (0.5095856189727783, 'at'),
 (0.509448230266571, 'on'),
 (0.5088790655136108, 'daughter'),
 (0.508336067199707, 'first'),
 (0.507088303565979, '1'),
 (0.5063262581825256, 'watch'),
 (0.506304919719696, 'in')]

Will be attempting vectors here.

In [ ]:
!pip install sister
import sister


In [ ]:
embedder = sister.MeanEmbedding(lang="en")

sentence = "I am a dog."
vector = embedder(sentence)  # 300-dim vector


Loading model...


In [ ]:
print(vector)

[ 1.45553529e-01  2.93777343e-02  7.25906417e-02 -3.32199186e-01
  1.16119541e-01  3.68818134e-01 -1.01360753e-01  6.13834858e-02
  9.52781364e-02  7.21023753e-02  2.20229030e-02 -1.12615541e-01
  5.77215105e-04  1.48535818e-01 -5.01247048e-02 -2.63762064e-02
 -1.00038767e-01  2.95178946e-02 -4.22474258e-02  6.19036630e-02
  8.73639509e-02  6.23530126e-04 -1.02920629e-01 -5.05673476e-02
 -2.43382186e-01 -5.01833446e-02  3.40687782e-02 -1.88491017e-01
  2.04292275e-02  1.45984618e-02  5.52195907e-02  8.40069652e-02
 -2.68052034e-02  1.21745087e-01 -1.02101944e-01  1.85941588e-02
 -7.98325688e-02 -2.78020445e-02 -2.32372917e-02  8.07202831e-02
  9.49704796e-02 -1.15997776e-01 -5.40836044e-02 -8.32328051e-02
  2.55872514e-02  1.02682985e-01 -1.13008637e-02 -7.92161524e-02
  5.31888008e-03 -5.63134067e-02  1.35553302e-02  2.82942466e-02
 -1.43566862e-01 -5.00163734e-02 -3.94095294e-02  7.77156502e-02
  9.01533887e-02  8.26578513e-02  1.37465566e-01  1.76570296e-01
 -1.79537944e-02  2.28038

In [ ]:
dfready = dforiginal.drop(['tweet_ids', 'news_url','id'], axis=1)

from string import punctuation
from nltk.corpus import stopwords




In [ ]:
def preprocess(data):
    """
    Preprocess data, filtering out stopwords, punctuation and lowering 
    all splitted tokens.
    
    :param data: the string data to be processed
    """    
    # Pad punctuation with spaces on both sides
    for char in ['.', '"', ',', '(', ')', '!', '?', ';', ':']:
        data = data.replace(char, ' ' + char + ' ')
    sw = stopwords.words('english')
    splitted_chunks = data.split()
    lowered_chunks = (item.lower() for item in splitted_chunks)
    chunks_without_punctuation = (chunk for chunk in lowered_chunks if chunk not in punctuation)
    chunks_without_stopwords = (chunk for chunk in chunks_without_punctuation if chunk not in sw)
    return " ".join(list(chunks_without_stopwords))


In [ ]:
import pandas as pd
#loop each row, text, fasttext embed, dump to file, get next : treat as a stream TRY THIS ALTERNATIVE!

def create_vectors(title):
  embedder = sister.MeanEmbedding(lang="en")
  sentence = title
  vector = embedder(sentence)  # 300-dim vector
  return vector

# executing the function
dfready['title'] = dfready['title'].apply(preprocess)

# executing the function
dfready['vectorcol'] = dfready['title'].apply(create_vectors)

Loading model...
Loading model...


Loading model...


Loading model...


Loading model...
Loading model...


Loading model...


...

Loading model...


Loading model...
Loading model...


Loading model...


Loading model...


KeyboardInterrupt: ignored

In [ ]:
dfready.head()